***LAB 6 : Implementation of Superdense Coding***

*STEP 1: Create Registers*

In [5]:
q = QuantumRegister(3, 'q')
c = ClassicalRegister(2, 'c')
qc = QuantumCircuit(q, c)

*STEP 2: Generate Random Bit*

In [6]:
qc_random = QuantumCircuit(q, c)
qc_random.h(q[0])

sim = AerSimulator()
qc_random.measure(q[0], 0)
job = sim.run(qc_random, shots=1)
result = job.result()
counts = result.get_counts()
random_bit = list(counts.keys())[0][0]
print(f" Random bit generated: {random_bit}")

 Random bit generated: 0


*STEP 3: Create Bell Pair between q1 (Alice) and q2 (Bob)*

In [7]:
qc = QuantumCircuit(q, c)
qc.h(q[1])
qc.cx(q[1], q[2])

*STEP 4: Alice encodes message (c, d)*

In [8]:
c_bit = randint(0, 1)
d_bit = randint(0, 1)
print(f" Alice's message bits: c = {c_bit}, d = {d_bit}")

if c_bit == 0 and d_bit == 0:
    pass
elif c_bit == 0 and d_bit == 1:
    qc.x(q[1])
elif c_bit == 1 and d_bit == 0:
    qc.z(q[1])
elif c_bit == 1 and d_bit == 1:
    qc.z(q[1])
    qc.x(q[1])

 Alice's message bits: c = 1, d = 1


*STEP 5: Bob decodes*

In [13]:
qc.cx(q[1], q[2])
qc.h(q[1])
qc.measure(q[1], c[0])
qc.measure(q[2], c[1])

*STEP 6: Run Simulation*

In [14]:
sim = AerSimulator()
compiled = transpile(qc, sim)
job = sim.run(compiled, shots=1024)
result = job.result()
counts = result.get_counts()

*STEP 7: Display Results*

In [15]:
print("\n Measurement results (Bob's side):")
print(counts)

measured_bits = max(counts, key=counts.get)
print(f"\n Bob received bits: {measured_bits[::-1]}")
print(f" Alice sent bits: {c_bit}{d_bit}")

if measured_bits[::-1] == f"{c_bit}{d_bit}":
    print("\n SUCCESS: Alice and Bob's bits match perfectly!")
else:
    print("\n Mismatch detected — check circuit design.")


 Measurement results (Bob's side):
{'01': 240, '11': 271, '10': 252, '00': 261}

 Bob received bits: 11
 Alice sent bits: 11

 SUCCESS: Alice and Bob's bits match perfectly!


*STEP 8: Visualize Circuit*

In [18]:
print("\n Quantum Circuit Layout:\n")
print(qc.draw(output='text'))


 Quantum Circuit Layout:

                                                                             »
q_0: ────────────────────────────────────────────────────────────────────────»
     ┌───┐     ┌───┐┌───┐     ┌───┐┌─┐     ┌───┐┌─┐     ┌───┐┌─┐     ┌───┐┌─┐»
q_1: ┤ H ├──■──┤ Z ├┤ X ├──■──┤ H ├┤M├──■──┤ H ├┤M├──■──┤ H ├┤M├──■──┤ H ├┤M├»
     └───┘┌─┴─┐└───┘└───┘┌─┴─┐└┬─┬┘└╥┘┌─┴─┐└┬─┬┘└╥┘┌─┴─┐└┬─┬┘└╥┘┌─┴─┐└┬─┬┘└╥┘»
q_2: ─────┤ X ├──────────┤ X ├─┤M├──╫─┤ X ├─┤M├──╫─┤ X ├─┤M├──╫─┤ X ├─┤M├──╫─»
          └───┘          └───┘ └╥┘  ║ └───┘ └╥┘  ║ └───┘ └╥┘  ║ └───┘ └╥┘  ║ »
c: 2/═══════════════════════════╩═══╩════════╩═══╩════════╩═══╩════════╩═══╩═»
                                1   0        1   0        1   0        1   0 »
«                  
«q_0: ─────────────
«          ┌───┐┌─┐
«q_1: ──■──┤ H ├┤M├
«     ┌─┴─┐└┬─┬┘└╥┘
«q_2: ┤ X ├─┤M├──╫─
«     └───┘ └╥┘  ║ 
«c: 2/═══════╩═══╩═
«            1   0 


Entanglement creation:
H(q1) then CX(q1,q2) — creates the Bell pair between Alice (q1) and Bob (q2).

Encoding by Alice:
Apply X and/or Z on q1 depending on bits d and c (so you see Z and/or X there).

Decoding by Bob:
CX(q1,q2) then H(q1) (then measure q1 and q2) — that recovers the two classical bits.

Random bit generator:
q0 is your extra qubit to generate randomness (Hadamard + measure). It’s logically separate from the encoding/decoding pair.